# Tweet sentiment analyses

## TODO

- (Embedding tweet instead of google news)
- Add OLE on embedding (+2 into dim)
- Add EV on embedding (+11 into dim)
- 
- Cross validation
- Comment vérifier l'overfitting?
- Jouer avec hyperparametres (dropout, fonction activation, nb de neuronnes, nb de layers, nb d'epoche / batch, optimisation, architecture, 

## Import libraries and files

In [1]:
import sys
import numpy as np
import numpy
import pandas as pd
import matplotlib as mpl

from preprocessing import data_preprocessing, data_preprocessing_test
from collections import Counter
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.utils.np_utils import to_categorical
from keras.layers import LSTM, Dropout, Dense, Bidirectional,  Flatten, Input, GRU
from keras.optimizers import Adam
from keras.models import load_model

#from preprocessing import standardization, data_preprocessing
#mpl.use('TkAgg')  # or whatever other backend that you want
#import matplotlib.pyplot as plt

np.random.seed(7)

Using TensorFlow backend.


## Getting data (word embedding and tweet)

In [2]:
EMBEDDING_FILE = "data/GoogleNews-vectors-negative300.bin"
EMBEDDING_DIM = 300

corpora_train_3 = "data/task_A/data_train_3.csv"
corpora_train_7 = "data/task_A/data_train_7.csv"

corpora_test_7 = "data/task_A/data_test_7.csv"

## Preprocessing

Example:

"Won the match #getin . Plus\u002c tomorrow is a very busy day\u002c with Awareness Day\u2019s and debates. Gulp. Debates..." ->

"win match plus tomorrow busy day awareness day debate gulp debate ..."

In [3]:
tweets_train_3, sentiments_train_3 = data_preprocessing(corpora_train_3, 'train')
tweets_train_7, sentiments_train_7 = data_preprocessing(corpora_train_7, 'test')

## Getting word index

Append the two list

In [4]:
all_tweet = tweets_train_3.append(tweets_train_7)

Tokenize tweets and get the word_index

In [5]:
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweet)
word_index = tokenizer.word_index

word_index

{'...': 1,
 'tomorrow': 2,
 'not': 3,
 'may': 4,
 'go': 5,
 'day': 6,
 'get': 7,
 'see': 8,
 'night': 9,
 'friday': 10,
 'time': 11,
 'like': 12,
 'sunday': 13,
 '1st': 14,
 'come': 15,
 'make': 16,
 'watch': 17,
 'saturday': 18,
 'one': 19,
 'u': 20,
 'say': 21,
 'game': 22,
 '2nd': 23,
 'new': 24,
 'want': 25,
 'monday': 26,
 'think': 27,
 'know': 28,
 'today': 29,
 'good': 30,
 'would': 31,
 '3rd': 32,
 'no': 33,
 'tonight': 34,
 'play': 35,
 'love': 36,
 '..': 37,
 'thursday': 38,
 'back': 39,
 'last': 40,
 'take': 41,
 'look': 42,
 'win': 43,
 'show': 44,
 'still': 45,
 'start': 46,
 'sun': 47,
 'best': 48,
 'first': 49,
 '️': 50,
 'wait': 51,
 'cannot': 52,
 'david': 53,
 'next': 54,
 'happy': 55,
 '4th': 56,
 'big': 57,
 'need': 58,
 'tuesday': 59,
 'work': 60,
 'great': 61,
 'year': 62,
 'live': 63,
 '☺': 64,
 'right': 65,
 'miss': 66,
 'world': 67,
 'morning': 68,
 'well': 69,
 'give': 70,
 'v': 71,
 'wednesday': 72,
 'ticket': 73,
 'gonna': 74,
 'hope': 75,
 'really': 76,
 'e

## Creating sequences

Example: "win match plus tomorrow busy day awareness day debate gulp debate ..." -> [43, 234, 769, 2, 1054, 6, 2896, 6, 424, 12301, 424, 1]

In [6]:
sequences_train_3 = tokenizer.texts_to_sequences(tweets_train_3)
sequences_train_7 = tokenizer.texts_to_sequences(tweets_train_7)

sequences = sequences_train_3 + sequences_train_7

## Calculate the max tweet sequence

In [7]:
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

print(MAX_SEQUENCE_LENGTH)

32


## Creating embedding matrix

In [8]:
nb_words = len(word_index) + 1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

In [9]:
print(embedding_matrix.shape)

(37785, 300)


Fill the embedded matrix with weight from embedding file or from saved file

In [11]:
def word2vec_load():
    try:
        word2vec = KeyedVectors.load("data/word2vect.w2v")
        print("a")
        return word2vec
    except:
        word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
        print("b")
        return word2vec

In [ ]:
word2vect = word2vec_load()

Create the oov randomly (oov: out of vocabulary)

In [22]:
oov = []
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0) # uniform distribution on [-1, 1]
oov = oov / np.linalg.norm(oov)

Fill the matrix with the word_embedding and oov if the word doesn't exist

In [23]:
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
    else:
        embedding_matrix[i] = oov

NameError: name 'word2vec' is not defined

## Padding data

In [24]:
data_train_3 = pad_sequences(sequences_train_3, maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequences_train_7, maxlen=MAX_SEQUENCE_LENGTH)

## Create one-hot vectors

In [25]:
indices_train_3 = np.arange(data_train_3.shape[0])
#data_train_3 = data_train_3[indices_train_3]

labels_train_3 = to_categorical(np.asarray(sentiments_train_3), 3)
#labels_train_3 = labels_train_3[indices_train_3]

labels_train_3

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [26]:
indices_train_7 = np.arange(data_train_7.shape[0])
#data_train_7 = data_train_7[indices_train_7]

labels_train_7 = to_categorical(np.asarray(sentiments_train_7), 7)
#labels_train_7 = labels_train_7[indices_train_7]

labels_train_7

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

## Splitting data

In [27]:
split_idx = int(len(data_train_3) * 0.70)

x_train_3, x_val_3 = data_train_3[:split_idx], data_train_3[split_idx:]
y_train_3, y_val_3 = labels_train_3 [:split_idx], labels_train_3[split_idx:]

print('training set: ' + str(len(x_train_3)) + ' samples')
print('validation set: ' + str(len(x_val_3)) + ' samples')
# print('test set: ' + str(len(x_test)) + ' samples')

print('x_train_3 shape: ', x_train_3.shape)
print('y_train_3 shape: ', y_train_3.shape)

training set: 35233 samples
validation set: 15100 samples
x_train_3 shape:  (35233, 32)
y_train_3 shape:  (35233, 3)


In [28]:
split_idx = int(len(data_train_7) * 0.85)

x_train_7, x_val_7 = data_train_7[:split_idx], data_train_7[split_idx:]
y_train_7, y_val_7 = labels_train_7 [:split_idx], labels_train_7[split_idx:]

print('training set: ' + str(len(x_train_7)) + ' samples')
print('validation set: ' + str(len(x_val_7)) + ' samples')

print('x_train_7 shape: ', x_train_7.shape)
print('y_train_7 shape: ', y_train_7.shape)

#, y_train_7, x_val_7,y_train_7)

training set: 1385 samples
validation set: 245 samples
x_train_7 shape:  (1385, 32)
y_train_7 shape:  (1385, 7)


## Creating embedding layer

In [29]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False, name='embedding_layer')

## Models definition

### Model 1

In [30]:
def model1(x_train_3, y_train_3, x_val_3, y_val_3, embedding_layer):

    model1 = Sequential()
    
    model1.add(embedding_layer)
    model1.add(LSTM(32))
    model1.add(Dropout(0.2))
    model1.add(Dense(32, activation='relu'))
    model1.add(Dropout(0.2))
    model1.add(Dense(3, activation='softmax'))
    
    model1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
    model1.summary()
    history = model1.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3), epochs=6, batch_size=50)
    
    model1.save("./model1.h5")

### Model 2

In [31]:
def model2(x_train_3, y_train_3, x_val_3, y_val_3, embedding_layer, epochs, batch_size):

    model2 = Sequential()
    
    model2.add(embedding_layer)
    model2.add(GRU(32))
    model2.add(Dropout(0.2))
    model2.add(Dense(3, activation='softmax'))

    model2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    model2.summary()
    history = model2.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3), epochs=6, batch_size=50)

    model2.save("./model2.h5")

## Call model

In [32]:
model1(x_train_3, y_train_3, x_val_3, y_val_3, embedding_layer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

## Transfert learning

Load model1

In [33]:
model = load_model("./model1.h5")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

Cut 2 last layers

In [34]:
model.layers.pop()
model.layers.pop()
#model.outputs = [model.layers[-1].output]

Create 3 layers

In [35]:
model.add(Dense(150, activation='relu', name='dense1'))
model.add(Dense(64, activation='relu', name='dense2'))
model.add(Dense(7, activation='softmax', name='dense3'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
history = model.fit(x_train_7, y_train_7, validation_data=(x_val_7, y_val_7), epochs=11, batch_size=50)
model.save("./model3.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
_________________________________________________________________
dense1 (Dense)               (None, 150)               600       
__________

/home/dadou/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 1385 samples, validate on 245 samples
Epoch 1/11
1385/1385 [==============================] - 2s 1ms/step - loss: 1.8802 - acc: 0.2664 - val_loss: 1.8654 - val_acc: 0.2327
Epoch 2/11
1385/1385 [==============================] - 1s 445us/step - loss: 1.8380 - acc: 0.2809 - val_loss: 1.8619 - val_acc: 0.2327
Epoch 3/11
1385/1385 [==============================] - 1s 435us/step - loss: 1.8359 - acc: 0.2809 - val_loss: 1.8605 - val_acc: 0.2327
Epoch 4/11
1385/1385 [==============================] - 1s 430us/step - loss: 1.8368 - acc: 0.2809 - val_loss: 1.8593 - val_acc: 0.2327
Epoch 5/11
1385/1385 [==============================] - 1s 426us/step - loss: 1.8350 - acc: 0.2809 - val_loss: 1.8670 - val_acc: 0.2327
Epoch 6/11
1385/1385 [==============================] - 1s 420us/step - loss: 1.8360 - acc: 0.2809 - val_loss: 1.8614 - val_acc: 0.2327
Epoch 7/11
1385/1385 [==============================] - 1s 438us/step - loss: 1.8356 - acc: 0.2809 - val_loss: 1.8640 - val_acc: 0.2327
Epo

## Testing new transfert model

In [36]:
tweets_test_7 = data_preprocessing_test(corpora_test_7)
sequences_test_7 = tokenizer.texts_to_sequences(tweets_test_7)
data_test_7 = pad_sequences(sequences_test_7, maxlen=MAX_SEQUENCE_LENGTH)

r = model.predict(data_test_7)
data = pd.read_csv("/home/abdou/Documents/datastories-semeval2017-task4-master/dataset/Subtask_A/gold/2018-Valence-oc-En-test-gold.txt", sep='\t', encoding='utf-8')

for i in range(len(r)):
    data['Intensity Class'][i]=["-1: slightly negative emotional state can be inferred",
                                "-2: moderately negative emotional state can be inferred",
                                "-3: very negative emotional state can be inferred",
                                "0: neutral or mixed emotional state can be inferred",
                                "1: slightly positive emotional state can be inferred",
                                "2: moderately positive emotional state can be inferred",
                                "3: very positive emotional state can be inferred"][numpy.argmax(r[i])]

FileNotFoundError: File b'data/task_A/data_test_7.csv' does not exist

## Write predicting results

In [ ]:
f = open("./submission_after.csv", "w")
f.write("ID	Tweet	Affect Dimension	Intensity Class\n")

for d in range(len(data)):
    f.write(data['ID'][d] + "\t" + data['Tweet'][d] + "\tvalence\t" + data['Intensity Class'][d] + "\n")

f.close()